In [1]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

## The following code is written according to the layout of UI and HTML layout of the page which dispalys the information for individual restaurants on OpenTable
## There is a if statement in place that only allows the first 300 reviews to be scrapped, however if the 300th review is on a new page, it will scrap the full page and therefore you can have more that 300 reviews in some conditions
## Two additional parameters "Cusine" and "Location" arr also being scrapped that.

In [ ]:
# Set up Chrome options
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # Can be 'eager' or 'none'

# Set up the WebDriver
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.opentable.ca/the-lobby-pinnacle-hotel-at-the-pier?corrid=096ba9c2-17b5-4fe5-a52a-ef84d26deca8&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-02-01T19%3A00%3A00')

Name = []
Date = []
Over_Rating = []
Food_Rating = []
Service_Rating = []
Ambience_Rating = []
Review = []
Cusine = []
Location = []

review_counter = 0


try:
    page_count = 0
    while True:  # Continue until the "Next" button can't be found
        # Increment page counter
        page_count += 1
        print(f"Page {page_count}")
        
        # Wait for the reviews section to load
        reviews_section = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="restProfileReviewsContent"]'))
        )
        
        # Find all individual review containers within the reviews section
        reviews = reviews_section.find_elements(By.CLASS_NAME, 'afkKaa-4T28-') 
        
        for review in reviews:
            if review_counter >= 300:  # Stop after scraping 300 reviews
                print("Reached 300 reviews.")
                break
        
        for review in reviews:
            # Extract the details from each review
            name = review.find_element(By.CSS_SELECTOR, 'section > p._1p30XHjz2rI-.C7Tp-bANpE4-').text
            date = review.find_element(By.CSS_SELECTOR, 'div > div.yuQYBV659bs-.C7Tp-bANpE4- > p').text
            overall_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(1) > span').text
            food_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(2) > span').text
            service_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(3) > span').text
            ambience_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(4) > span').text
            review_text = review.find_element(By.CSS_SELECTOR, 'div > div._6rFG6U7PA6M- > span:nth-child(1)').text
            cusine = review.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[1]/section[2]/div[5]/section[1]/div/div/div/div/ul[1]/li[3]/div/div/div').text
            location = review.find_element(By.XPATH, '//*[@id="mainContent"]/div/div[2]/div[1]/section[2]/div[5]/section[1]/div/div/div/div/ul[2]/li[1]/div/div/div/a').text
            
            
            Name.append(name)
            Date.append(date)
            Over_Rating.append(overall_rating)
            Food_Rating.append(food_rating)
            Service_Rating.append(service_rating)
            Ambience_Rating.append(ambience_rating)
            Review.append(review_text)
            Cusine.append(cusine)
            Location.append(location)
            
            # Print out the extracted information
            #print("Name: " + name)
            #print("Date: " + date)
            #print("Overall Rating: " + overall_rating)
            #print("Food Rating: " + food_rating)
            #print("Service Rating: " + service_rating)
            #print("Ambience Rating: " + ambience_rating)
            #print("Review: " + review_text + "\n")
            review_counter += 1

        if review_counter >= 300:
            break
            
            
            
        # Wait for the "Next" button to be clickable
        wait = WebDriverWait(driver, 20)
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviews"]/section/footer/div/div[2]/a')))
            next_button.click()
        except (TimeoutException, NoSuchElementException):
            # "Next" button is not present or not clickable, exit the loop
            print("Reached the last page or the next button is not clickable.")
            break
        except StaleElementReferenceException:
            # The reference to the button is stale, re-find the button
            continue

        # Wait for a specific element that indicates the new page has loaded
        # Replace 'unique_element_xpath_on_new_page' with the actual XPATH
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviews"]/section/footer/div/div[2]/a'))
        )

except Exception as e:
    # Handle any other exceptions
    print("An error occurred:", e)
finally:
    # Close the browser when done
    driver.quit()
    
    
    df= pd.DataFrame(
    {'Name': Name,
     'Date': Date,
     'Cusine' : Cusine,
     'Overall_rating': Over_Rating,
     'Food_rating': Food_Rating,
     'Service_rating': Service_Rating,
     'Ambience_Rating': Ambience_Rating,
     'Review': Review,
     'Location' : Location})

# Add a new column with the same value for all rows
df['Restaurant'] = 'The Lobby'


df.to_csv("C:/Users/12367/Documents/Python Scripts/Open Table Review.csv", index=False, mode='a', header= False)

## The following code is written according to the layout of UI and HTML layout of the page which dispalys the information for individual restaurants on OpenTable
## The following code will execute till all the reviews are scrapped, depending on the computational power, I will suggest to move the statements for adding the data to a dataframe and saving it to a .csv file to be mode to a new block of code and to be executed seperately

In [12]:
# Set up Chrome options
chrome_options = Options()
chrome_options.page_load_strategy = 'eager'  # Can be 'eager' or 'none'

# Set up the WebDriver
driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.opentable.ca/homer-street-cafe-and-bar?corrid=f258c8b1-5951-4a15-9b29-ffa9a838ec23&avt=eyJ2IjoyLCJtIjoxLCJwIjowLCJzIjowLCJuIjowfQ&p=2&sd=2024-02-01T19%3A00%3A00')

Name = []
Date = []
Over_Rating = []
Food_Rating = []
Service_Rating = []
Ambience_Rating = []
Review = []


try:
    page_count = 0
    while True:  # Continue until the "Next" button can't be found
        # Increment page counter
        page_count += 1
        print(f"Page {page_count}")
        
        # Wait for the reviews section to load
        reviews_section = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="restProfileReviewsContent"]'))
        )
        
        # Find all individual review containers within the reviews section
        reviews = reviews_section.find_elements(By.CLASS_NAME, 'afkKaa-4T28-') 
        
        for review in reviews:
            # Extract the details from each review
            name = review.find_element(By.CSS_SELECTOR, 'section > p._1p30XHjz2rI-.C7Tp-bANpE4-').text
            date = review.find_element(By.CSS_SELECTOR, 'div > div.yuQYBV659bs-.C7Tp-bANpE4- > p').text
            overall_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(1) > span').text
            food_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(2) > span').text
            service_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(3) > span').text
            ambience_rating = review.find_element(By.CSS_SELECTOR, 'div > ol > li:nth-child(4) > span').text
            review_text = review.find_element(By.CSS_SELECTOR, 'div > div._6rFG6U7PA6M- > span:nth-child(1)').text
            
            Name.append(name)
            Date.append(date)
            Over_Rating.append(overall_rating)
            Food_Rating.append(food_rating)
            Service_Rating.append(service_rating)
            Ambience_Rating.append(ambience_rating)
            Review.append(review_text)
            
            # Print out the extracted information
            #print("Name: " + name)
            #print("Date: " + date)
            #print("Overall Rating: " + overall_rating)
            #print("Food Rating: " + food_rating)
            #print("Service Rating: " + service_rating)
            #print("Ambience Rating: " + ambience_rating)
            #print("Review: " + review_text + "\n")
        
        # Wait for the "Next" button to be clickable
        wait = WebDriverWait(driver, 20)
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviews"]/section/footer/div/div[2]/a')))
            next_button.click()
        except (TimeoutException, NoSuchElementException):
            # "Next" button is not present or not clickable, exit the loop
            print("Reached the last page or the next button is not clickable.")
            break
        except StaleElementReferenceException:
            # The reference to the button is stale, re-find the button
            continue

        # Wait for a specific element that indicates the new page has loaded
        # Replace 'unique_element_xpath_on_new_page' with the actual XPATH
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="reviews"]/section/footer/div/div[2]/a'))
        )

except Exception as e:
    # Handle any other exceptions
    print("An error occurred:", e)
finally:
    # Close the browser when done
    driver.quit()
    
    
    df= pd.DataFrame(
    {'Name': Name,
     'Date': Date,
     'Overall_rating': Over_Rating,
     'Food_rating': Food_Rating,
     'Service_rating': Service_Rating,
     'Ambience_Rating': Ambience_Rating,
     'Review': Review})

# Add a new column with the same value for all rows
df['Restaurant'] = 'Homer Street Cafe & Bar'


df.to_csv("C:/Users/12367/Documents/Python Scripts/Miku Restaurant Open Table Review.csv", mode='a', header=False, index=False)

Page 1
Page 2
Page 3
Page 4
An error occurred: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6E7525E42+3538674]
	(No symbol) [0x00007FF6E7144C02]
	(No symbol) [0x00007FF6E6FF5AEB]
	(No symbol) [0x00007FF6E703BF4E]
	(No symbol) [0x00007FF6E703C0CC]
	(No symbol) [0x00007FF6E707E477]
	(No symbol) [0x00007FF6E705F0EF]
	(No symbol) [0x00007FF6E707BDE3]
	(No symbol) [0x00007FF6E705EE53]
	(No symbol) [0x00007FF6E702F514]
	(No symbol) [0x00007FF6E7030631]
	GetHandleVerifier [0x00007FF6E7556CAD+3738973]
	GetHandleVerifier [0x00007FF6E75AC506+4089270]
	GetHandleVerifier [0x00007FF6E75A4823+4057299]
	GetHandleVerifier [0x00007FF6E7275C49+720121]
	(No symbol) [0x00007FF6E715126F]
	(No symbol) [0x00007FF6E714C304]
	(No symbol) [0x00007FF6E714C432]
	(No symbol) [0x00007FF6E713BD04]
	BaseThreadInitThunk [0x00007FF8DAB07344+20]
	RtlUserThreadStart [0x00007FF8DBB626B1+33]

